In [3]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
import matplotlib.pylab as plt
from os import listdir
import pandas as pd
from PIL import Image
from matplotlib import image
from matplotlib import pyplot
import numpy

In [4]:
## from keras import backend as K
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.optimizers import Adam
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.preprocessing import image_dataset_from_directory

In [ ]:
train_x = []
train_y = []
for p in ['buy', 'sell', 'nothing']:
    path = f'stock_graphs/train/{p}/'
    for file in [f for f in listdir(path)]:
        train_x.append(image.imread(path+file))
        train_y.append(p)

In [ ]:
test_x = []
test_y = []
for p in ['buy', 'sell', 'nothing']:
    path = f'stock_graphs/test/{p}/'
    for file in [f for f in listdir(path)]:
        test_x.append(image.imread(path+file))
        test_y.append(p)

In [ ]:
np.array(train_x).shape

In [ ]:
x.shape[1:]

In [ ]:
import gc; gc.collect()

In [20]:
batch_size = 16
image_size = (862, 1200, 3)

train_generator = image_dataset_from_directory(
        'stock_graphs/train',  # this is the target directory
        image_size=image_size[:-1],  # all images will be resized to 150x150
        batch_size=batch_size,
        label_mode='categorical',)  

# this is a similar generator, for validation data
validation_generator = image_dataset_from_directory(
        './stock_graphs/test',
        image_size=image_size[:-1],
        batch_size=batch_size,
        label_mode='categorical',)

Found 7577 files belonging to 3 classes.
Found 1758 files belonging to 3 classes.


In [21]:
# CNN model
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=image_size))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', 'mae', 'mse'])

In [22]:
model.fit_generator(
    train_generator,
        steps_per_epoch=2000 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=800 // batch_size)
model.save_weights('second_try.h5')  # always save your weights after training or during training

Epoch 1/50


ResourceExhaustedError:  OOM when allocating tensor with shape[16,858,1196,64] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[node sequential_3/conv2d_5/Relu (defined at <ipython-input-22-31e0f140470d>:6) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1612]

Function call stack:
train_function


In [24]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 16050779726139915175
]


In [25]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0
